# End-to-End Pipeline Example

This notebook demonstrates the complete MediClaimGPT pipeline from raw medical claims to trained classifiers.

## Overview
- **Input**: Raw medical claims CSV file
- **Process**: Generate embeddings → Train classifiers → Evaluate models
- **Output**: Embeddings, trained models, performance metrics, and reports

## Pipeline Flow
1. **Data Loading**: Load and preprocess medical claims
2. **Embedding Generation**: Generate embeddings using MediClaimGPT API
3. **Classification Training**: Train multiple binary classifiers
4. **Model Evaluation**: Evaluate and compare model performance
5. **Results Summary**: Generate comprehensive reports

## Prerequisites
- MediClaimGPT API server running on `http://localhost:8000`
- Medical claims dataset in CSV format

## Setup and Imports

In [1]:
import sys
import os
import pandas as pd
import numpy as np
from pathlib import Path
import json
import requests
import time

# Add project root to path
project_root = Path().cwd().parent
sys.path.append(str(project_root))

from pipelines.end_to_end_pipeline import EndToEndPipeline
from models.config_models import PipelineConfig
from models.pipeline_models import PipelineJob
from utils.logging_utils import get_logger

print(f"Project root: {project_root}")
print(f"Working directory: {os.getcwd()}")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Project root: /home/kosaraju/mgpt_eval
Working directory: /home/kosaraju/mgpt_eval/examples


## API Server Health Check

In [2]:
# Check if MediClaimGPT API is running
api_url = "http://localhost:8000"

def check_api_health(base_url, timeout=5):
    """Check if API server is running and responsive."""
    try:
        # Try to reach the health endpoint or any basic endpoint
        response = requests.get(f"{base_url}/docs", timeout=timeout)
        return response.status_code == 200
    except requests.exceptions.RequestException:
        return False

# Test API connectivity
api_available = check_api_health(api_url)
print(f"API Server Status: {'✅ Available' if api_available else '❌ Not Available'}")
print(f"API URL: {api_url}")

if not api_available:
    print("\n⚠️ Warning: API server is not available!")
    print("Please start the MediClaimGPT API server before running this pipeline.")
    print("The pipeline will fail without a running API server.")
else:
    print("\n✅ API server is ready for embedding generation!")

API Server Status: ✅ Available
API URL: http://localhost:8000

✅ API server is ready for embedding generation!


## Configuration Setup

In [3]:
# Load end-to-end configuration
config_path = "configs/end_to_end_example_config.yaml"
config = PipelineConfig.from_yaml(config_path)

print("Configuration loaded successfully!")
print(f"Job name: {config.job.name}")
print(f"Dataset path: {config.input.dataset_path}")
print(f"Output directory: {config.job.output_dir}")
print(f"\nPipeline stages enabled:")
for stage, enabled in config.pipeline_stages.model_dump().items():
    status = "✅" if enabled else "❌"
    print(f"  {status} {stage}")

print(f"\nClassifiers to train: {config.classification.models}")
print(f"API base URL: {config.model_api.base_url}")
print(f"Embedding batch size: {config.embedding_generation.batch_size}")

Configuration loaded successfully!
Job name: end_to_end_pipeline_example
Dataset path: /home/kosaraju/mgpt_eval/examples/data/medical_claims_complete.csv
Output directory: /home/kosaraju/mgpt_eval/examples/outputs

Pipeline stages enabled:
  ✅ embeddings
  ✅ classification
  ✅ evaluation
  ❌ target_word_eval
  ✅ summary_report
  ✅ method_comparison

Classifiers to train: ['logistic_regression', 'svm', 'random_forest']
API base URL: http://localhost:8000
Embedding batch size: 16


## Input Data Verification

In [4]:
# Check input dataset
dataset_path = Path(config.input.dataset_path)
print(f"Dataset path: {dataset_path}")
print(f"Dataset exists: {dataset_path.exists()}")

if dataset_path.exists():
    # Load and inspect the dataset
    df = pd.read_csv(dataset_path)
    print(f"\nDataset shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    
    # Check for required columns
    required_cols = ['mcid', 'claim', 'label']
    missing_cols = [col for col in required_cols if col not in df.columns]
    
    if missing_cols:
        print(f"\n❌ Missing required columns: {missing_cols}")
    else:
        print(f"\n✅ All required columns present: {required_cols}")
        
        # Show data statistics
        print(f"\nData statistics:")
        print(f"  Total claims: {len(df)}")
        print(f"  Unique MCIDs: {df['mcid'].nunique()}")
        print(f"  Label distribution:")
        print(f"    {df['label'].value_counts().to_dict()}")
        
        # Show sample data
        print(f"\nSample data:")
        print(df.head(3).to_string(max_colwidth=50))
else:
    print("\n❌ Dataset file not found!")
    print("Please ensure the dataset exists or update the config file.")

Dataset path: /home/kosaraju/mgpt_eval/examples/data/medical_claims_complete.csv
Dataset exists: True

Dataset shape: (50, 3)
Columns: ['mcid', 'claims', 'label']

❌ Missing required columns: ['claim']


## Initialize End-to-End Pipeline

In [5]:
# Setup logging
logger = get_logger("end_to_end_pipeline", config.logging)

# Create job configuration from the pipeline config
job_config = PipelineJob(
    dataset_path=config.input.dataset_path,
    job_name=config.job.name
)

# Initialize end-to-end pipeline
pipeline = EndToEndPipeline(config, job_config)

print("End-to-end pipeline initialized successfully!")
print(f"Pipeline will process: {config.input.dataset_path}")
print(f"Results will be saved to: {config.job.output_dir}")

End-to-end pipeline initialized successfully!
Pipeline will process: /home/kosaraju/mgpt_eval/examples/data/medical_claims_complete.csv
Results will be saved to: /home/kosaraju/mgpt_eval/examples/outputs


## Run End-to-End Pipeline

This will execute all enabled pipeline stages in sequence:
1. **Embedding Generation**: Process claims through MediClaimGPT API
2. **Classification Training**: Train binary classifiers on embeddings
3. **Model Evaluation**: Evaluate trained models
4. **Report Generation**: Create comprehensive performance reports

In [6]:
# Run the complete end-to-end pipeline
start_time = time.time()

try:
    print("Starting end-to-end pipeline execution...")
    print("This may take several minutes depending on dataset size and API response time.\n")
    
    results = pipeline.run()
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    print(f"\n🎉 End-to-end pipeline completed successfully!")
    print(f"⏱️  Total execution time: {execution_time:.2f} seconds ({execution_time/60:.1f} minutes)")
    print(f"📊 Results: {results}")
    
except Exception as e:
    end_time = time.time()
    execution_time = end_time - start_time
    
    print(f"\n❌ Pipeline execution failed after {execution_time:.2f} seconds")
    print(f"Error: {e}")
    
    import traceback
    print("\nFull error traceback:")
    traceback.print_exc()

Starting end-to-end pipeline execution...
This may take several minutes depending on dataset size and API response time.

2025-06-04 00:28:00,090 - end_to_end_pipeline - ERROR - Configuration validation failed: Target word evaluation enabled but no target words specified

❌ Pipeline execution failed after 0.03 seconds
Error: Configuration validation failed: Target word evaluation enabled but no target words specified

Full error traceback:


Traceback (most recent call last):
  File "/tmp/ipykernel_126133/1660034385.py", line 8, in <module>
    results = pipeline.run()
  File "/home/kosaraju/mgpt_eval/pipelines/end_to_end_pipeline.py", line 386, in run
    raise ValueError(error_msg)
ValueError: Configuration validation failed: Target word evaluation enabled but no target words specified


## Results Overview

In [7]:
# Analyze generated outputs
output_dir = Path(config.job.output_dir)

# The EndToEndPipeline creates a job-specific subdirectory
job_output_dir = output_dir / config.job.name
embeddings_dir = job_output_dir / "embeddings"
models_dir = job_output_dir / "models"
metrics_dir = job_output_dir / "metrics"
logs_dir = output_dir / "logs"

print("🔍 PIPELINE OUTPUTS SUMMARY")
print("=" * 50)
print(f"Job output directory: {job_output_dir}")

# Embeddings
if embeddings_dir.exists():
    embedding_files = list(embeddings_dir.glob("*.csv")) + list(embeddings_dir.glob("*.json"))
    print(f"\n📊 Embeddings ({len(embedding_files)} files):")
    for file in embedding_files:
        if file.suffix == '.csv':
            try:
                df = pd.read_csv(file)
                print(f"  - {file.name}: {df.shape[0]} embeddings")
            except:
                print(f"  - {file.name}: (could not read)")
        else:
            print(f"  - {file.name}")
else:
    print(f"\n❌ No embeddings found in {embeddings_dir}")

# Models
if models_dir.exists():
    model_dirs = [d for d in models_dir.iterdir() if d.is_dir()]
    model_files = list(models_dir.glob("*.pkl"))
    print(f"\n🤖 Trained Models:")
    print(f"  Model directories: {len(model_dirs)}")
    print(f"  Model files: {len(model_files)}")
    
    for model_dir in model_dirs:
        model_files_in_dir = list(model_dir.glob("*.pkl"))
        metric_files_in_dir = list(model_dir.glob("*.json"))
        print(f"    - {model_dir.name}: {len(model_files_in_dir)} models, {len(metric_files_in_dir)} metrics")
    
    for model_file in model_files:
        print(f"    - {model_file.name}")
else:
    print(f"\n❌ No models found in {models_dir}")

# Metrics
if metrics_dir.exists():
    metric_files = list(metrics_dir.glob("*.json"))
    csv_files = list(metrics_dir.glob("*.csv"))
    print(f"\n📈 Evaluation Results:")
    print(f"  - Metric files: {len(metric_files)}")
    print(f"  - CSV reports: {len(csv_files)}")
    for file in metric_files + csv_files:
        print(f"    • {file.name}")
else:
    print(f"\n❌ No metrics found in {metrics_dir}")

# Logs
if logs_dir.exists():
    log_files = list(logs_dir.glob("*.log")) + list(logs_dir.glob("*.json"))
    print(f"\n📝 Log Files: {len(log_files)}")
    for file in log_files:
        print(f"  - {file.name}")
else:
    print(f"\n❌ No logs found in {logs_dir}")

🔍 PIPELINE OUTPUTS SUMMARY
Job output directory: /home/kosaraju/mgpt_eval/examples/outputs/end_to_end_pipeline_example

📊 Embeddings (0 files):

🤖 Trained Models:
  Model directories: 0
  Model files: 0

📈 Evaluation Results:
  - Metric files: 0
  - CSV reports: 0

📝 Log Files: 6
  - embedding_pipeline.log
  - end_to_end_pipeline.log
  - classification_pipeline.log
  - end_to_end_pipeline.json
  - classification_pipeline.json
  - embedding_pipeline.json


## Model Performance Analysis

In [8]:
# Load and compare model performance
output_dir = Path(config.job.output_dir)
job_output_dir = output_dir / config.job.name
models_dir = job_output_dir / "models"

if models_dir.exists():
    model_dirs = [d for d in models_dir.iterdir() if d.is_dir()]
    model_files = list(models_dir.glob("*.pkl"))
    
    # Check for metrics in both subdirectories and root models directory
    all_metric_files = list(models_dir.glob("*.json"))
    for model_dir in model_dirs:
        all_metric_files.extend(list(model_dir.glob("*.json")))
    
    performance_data = []
    
    print("🏆 MODEL PERFORMANCE COMPARISON")
    print("=" * 60)
    
    for metrics_file in all_metric_files:
        if '_metrics_' in metrics_file.name:
            try:
                with open(metrics_file, 'r') as f:
                    metrics = json.load(f)
                
                # Extract classifier type from directory name or filename
                if metrics_file.parent != models_dir:
                    # File is in a subdirectory
                    classifier_type = metrics_file.parent.name.split('_')[-1]
                else:
                    # File is in root models directory
                    classifier_type = metrics_file.name.split('_metrics_')[0]
                
                # Get test metrics
                test_metrics = metrics.get('test_metrics', {})
                
                performance = {
                    'Classifier': classifier_type.replace('_', ' ').title(),
                    'Accuracy': test_metrics.get('accuracy', 'N/A'),
                    'Precision': test_metrics.get('precision', 'N/A'),
                    'Recall': test_metrics.get('recall', 'N/A'),
                    'F1 Score': test_metrics.get('f1_score', 'N/A'),
                    'ROC AUC': test_metrics.get('roc_auc', 'N/A')
                }
                
                performance_data.append(performance)
                
                # Display individual model performance
                print(f"\n{performance['Classifier'].upper()}:")
                for metric, value in performance.items():
                    if metric != 'Classifier':
                        if isinstance(value, float):
                            print(f"  {metric:12}: {value:.4f}")
                        else:
                            print(f"  {metric:12}: {value}")
                            
            except Exception as e:
                print(f"\n❌ Error loading metrics for {metrics_file.name}: {e}")
    
    # Create comparison table
    if performance_data:
        comparison_df = pd.DataFrame(performance_data)
        
        print("\n" + "=" * 80)
        print("📊 COMPREHENSIVE PERFORMANCE COMPARISON")
        print("=" * 80)
        
        # Format numeric columns for better display
        numeric_cols = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC']
        for col in numeric_cols:
            if col in comparison_df.columns:
                comparison_df[col] = comparison_df[col].apply(
                    lambda x: f"{x:.4f}" if isinstance(x, float) else str(x)
                )
        
        print(comparison_df.to_string(index=False))
        
        # Find best performing model for each metric
        print("\n🥇 BEST PERFORMERS:")
        for col in numeric_cols:
            if col in comparison_df.columns:
                # Convert back to float for comparison
                numeric_values = pd.to_numeric(comparison_df[col], errors='coerce')
                if not numeric_values.isna().all():
                    best_idx = numeric_values.idxmax()
                    best_model = comparison_df.loc[best_idx, 'Classifier']
                    best_value = comparison_df.loc[best_idx, col]
                    print(f"  {col:12}: {best_model} ({best_value})")
    else:
        print("No model metrics found in the expected format.")
                    
else:
    print("❌ No model performance data available.")
    print("The classification stage may not have completed successfully.")
    print(f"Expected models directory: {models_dir}")

🏆 MODEL PERFORMANCE COMPARISON
No model metrics found in the expected format.


## Data and Processing Insights

In [9]:
# Analyze embedding quality and processing statistics
print("🔬 PROCESSING INSIGHTS")
print("=" * 50)

# Embedding statistics
if embeddings_dir.exists():
    embedding_files = list(embeddings_dir.glob("*.csv"))
    
    for file in embedding_files:
        df = pd.read_csv(file)
        
        print(f"\n📊 Embeddings Analysis - {file.name}:")
        print(f"  Total samples: {len(df)}")
        
        if 'label' in df.columns:
            label_dist = df['label'].value_counts()
            print(f"  Label distribution: {label_dist.to_dict()}")
            print(f"  Class balance: {label_dist.min() / label_dist.max():.3f}")
        
        # Analyze embedding dimensions
        embedding_cols = [col for col in df.columns if col.startswith('dim_') or col.isdigit()]
        if not embedding_cols:
            # Try to find embedding columns differently
            non_meta_cols = [col for col in df.columns if col not in ['mcid', 'label', 'claim']]
            if non_meta_cols:
                embedding_cols = non_meta_cols
        
        if embedding_cols:
            embeddings = df[embedding_cols].astype(float)
            print(f"  Embedding dimensions: {len(embedding_cols)}")
            print(f"  Mean embedding norm: {np.linalg.norm(embeddings.values, axis=1).mean():.4f}")
            print(f"  Embedding variance: {embeddings.var().mean():.6f}")

# Processing time analysis from logs
if logs_dir.exists():
    log_files = list(logs_dir.glob("*.json"))
    
    for log_file in log_files:
        try:
            with open(log_file, 'r') as f:
                log_data = [json.loads(line) for line in f if line.strip()]
            
            print(f"\n⏱️  Processing Time Analysis - {log_file.name}:")
            
            # Look for timing information in logs
            start_times = [log for log in log_data if 'started' in log.get('message', '').lower()]
            end_times = [log for log in log_data if 'completed' in log.get('message', '').lower()]
            
            if start_times and end_times:
                print(f"  Pipeline stages logged: {len(start_times)} starts, {len(end_times)} completions")
            
            # Count different log levels
            log_levels = [log.get('levelname', 'UNKNOWN') for log in log_data]
            level_counts = pd.Series(log_levels).value_counts()
            print(f"  Log level distribution: {level_counts.to_dict()}")
            
        except Exception as e:
            print(f"  Error analyzing {log_file.name}: {e}")

print("\n✅ Analysis complete!")

🔬 PROCESSING INSIGHTS

⏱️  Processing Time Analysis - end_to_end_pipeline.json:
  Log level distribution: {'UNKNOWN': 1}

⏱️  Processing Time Analysis - classification_pipeline.json:
  Log level distribution: {}

⏱️  Processing Time Analysis - embedding_pipeline.json:
  Log level distribution: {'UNKNOWN': 7}

✅ Analysis complete!


## Next Steps and Recommendations

After running the complete end-to-end pipeline:

### 🎯 **Model Selection**
- Review the performance comparison above
- Choose the best model based on your primary metric (accuracy, precision, recall, F1, or AUC)
- Consider the trade-offs between different metrics for your use case

### 🔧 **Performance Optimization**
1. **Hyperparameter Tuning**: Adjust the search grids in the config for better performance
2. **Feature Engineering**: Consider additional preprocessing or feature extraction
3. **Data Augmentation**: Add more training data if available
4. **Ensemble Methods**: Combine multiple models for improved performance

### 📊 **Data Analysis**
- Analyze misclassified examples to understand model limitations
- Check for class imbalance and consider resampling techniques
- Examine embedding quality and clustering patterns

### 🚀 **Production Deployment**
- Save the best performing model for production use
- Set up model monitoring and performance tracking
- Implement A/B testing for model updates

### 📈 **Scaling Considerations**
- For larger datasets, consider increasing API batch sizes
- Implement parallel processing for embedding generation
- Use distributed training for very large datasets

## Configuration Customization

To adapt this pipeline for your specific needs:

1. **Data Source**: Update `dataset_path` to point to your medical claims data
2. **API Configuration**: Adjust `model_api` settings for your deployment
3. **Classifier Selection**: Modify the `models` list to include/exclude classifiers
4. **Hyperparameters**: Customize search grids for each classifier
5. **Evaluation Metrics**: Add/remove metrics based on your requirements
6. **Output Structure**: Adjust output directories and file formats

## Troubleshooting

Common issues and solutions:

- **API Connection Issues**: Ensure MediClaimGPT server is running and accessible
- **Memory Errors**: Reduce batch sizes or process data in smaller chunks
- **Performance Issues**: Adjust hyperparameter grids or try different classifiers
- **Data Format Errors**: Verify CSV format and required columns (mcid, claim, label)

## Files Generated

This complete pipeline generates:
- **Embeddings**: Train/test embedding files (.csv)
- **Models**: Trained classifier models (.pkl files)
- **Metrics**: Performance metrics and evaluation results (JSON/CSV)
- **Logs**: Detailed execution logs for debugging
- **Reports**: Comprehensive performance comparison reports